# Preprocessing

In [2]:
import pandas as pd
import numpy as np
import sys

### Step 1
Write a function that processes a single .txt file. It must:
* drop rows that do not contain a unique school identifier.
* drop rows that correspond to elementary/middle school education. We are focusing on high school data.

In [3]:
def read_text_file(textfilepath):
    """
    Input: textfilepath, a path to the text file to be generated to a datafrmae
    Output: Pandas DataFrame corresponding to input text file
    """
    df = pd.read_csv(textfilepath, sep="\t", encoding="ISO-8859-1")
    return df 

In [4]:
df1 = read_text_file("./txt_files/absenteeism_txt_files/2016-17_ChronAbsenteeism.txt")
df2 = read_text_file("./txt_files/absenteeism_txt_files/2017-18_ChronAbsenteeism.txt")

In [5]:
def drop_rows(df):
    """
    Input: Pandas DataFrame
    Output: Pandas DataFrame with (a) rows with no school code and (b) rows corresponding to elementary/
            middle school education removed
    """
    rows_to_drop = []
    #drop rows that do not have unique school code
    for i, code in enumerate(df["SchoolCode"]):
        if pd.isnull(code) or code in [0,1]:
            rows_to_drop.append(i)
    #drop rows that correspond to elementary and middle school data
    for i, reporting_category in enumerate(df["ReportingCategory"]):
        if reporting_category in ["GRKN", "GRK", "GR13", "GR46", "GR78", "GRK8", "GR912", "GRUG"]:
            rows_to_drop.append(i)
    df = df.drop(rows_to_drop,axis=0)
    return df

In [6]:
df1 = drop_rows(df1)
df2 = drop_rows(df2)

### Step 2
We only want one row per school. As of now, the DataFrame has multiple rows per school to give metrics across different reporting groups.

To do this, we will construct of matrix where rows are individual schools and columns are 
["RB", "RI", "RA", "RF", "RD", "RP", "RT", "RW", "GM", "GF", "GX", "GZ", "SE", "SD", "SS", "SM", "SF", "SH", "TA"]

These are each of the reporting categories.

In [7]:
def generate_column_index_mapping(columns):
    mapping = {}
    for i,col in enumerate(columns):
        mapping[col] = i
    return mapping
mapping = generate_column_index_mapping(["RB", "RI", "RA", "RF", "RH", "RD", "RP", "RT", "RW", "GM", "GF", "GX", "GZ", "SE", "SD", "SS", "SM", "SF", "SH", "TA"])

In [8]:
all_columns = ["AcademicYear", "AggregateLevel", "CountryCode", "DistrictCode", "SchoolCode", "CountyName",
          "DistrictName", "SchoolName", "CharterYN", "CAR_RB", "CAR_RI", "CAR_RA", "CAR_RF", "CAR_RH", "CAR_RD",
          "CAR_RP", "CAR_RT", "CAR_RW", "CAR_GM", "CAR_GF", "CAR_GX", "CAR_GZ", "CAR_SE", "CAR_SD", "CAR_SS",
          "CAR_SM", "CAR_SF", "CAR_SH", "CAR_TA"]
reporting_category_columns = ["CAR_RB", "CAR_RI", "CAR_RA", "CAR_RF", "CAR_RH", "CAR_RD",
          "CAR_RP", "CAR_RT", "CAR_RW", "CAR_GM", "CAR_GF", "CAR_GX", "CAR_GZ", "CAR_SE", "CAR_SD", "CAR_SS",
          "CAR_SM", "CAR_SF", "CAR_SH", "CAR_TA"]

In [9]:
print(df1.shape)
print(df2.shape)

(151396, 13)
(151101, 13)


In [10]:
def colapse_df(df):
    #determine which rows are associated with each school
    school_codes = df["SchoolCode"].unique()
    associated_rows = {}
    for code in school_codes:
        associated_rows[code] = []
    for i in range(df.shape[0]):
        #df["SchoolCode"][df.index[i]] = school code at row i of df
        associated_rows[df["SchoolCode"][df.index[i]]].append(i)
        
    array = np.resize(np.array(all_columns), (1,len(all_columns))) #resize to stack rows with numpy
    for code in school_codes:
        rows = associated_rows[code] #get rows associated with code
        firstrow = df.iloc[rows[0]] #use first row to capture redundant data
        school_data = np.array([[firstrow["AcademicYear"], firstrow["AggregateLevel"], firstrow["CountyCode"], 
                               firstrow["DistrictCode"], firstrow["SchoolCode"], firstrow["CountyName"],
                               firstrow["DistrictName"], firstrow["SchoolName"], firstrow["CharterYN"],
                                       None,None,None,None,None,None,None,None,None,None,None,None,None,
                                       None,None,None,None,None,None,None]])
        #note that all reporting categories data is initialized as None so we can determine what was missing
        for row in rows:
            #write reporting category data to correct column
            school_data[0][9+mapping[df.iloc[row]["ReportingCategory"]]] = df.iloc[row]["ChronicAbsenteeismRate"]
        array = np.append(array,school_data,axis=0) #stack rows
    return pd.DataFrame(array[1:,:], columns=all_columns) #convert to df and return 

In [11]:
new_df1 = colapse_df(df1)
new_df2 = colapse_df(df2)

In [12]:
print(new_df1.shape)
print(new_df2.shape)

(10016, 29)
(10012, 29)


In [13]:
rows = []
for i in range(df1.shape[0]):
    if df1["SchoolCode"][df1.index[i]] == 112607:
        rows.append(i)
df1.iloc[rows]

,AcademicYear,AggregateLevel,CountyCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,ReportingCategory,ChronicAbsenteeismEligibleCumula,ChronicAbsenteeismCount,ChronicAbsenteeismRate
53047,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,GF,221.0,58.0,26.2
53056,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,GM,194.0,41.0,21.1
53098,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RA,NaN,NaN,NaN
53106,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RB,153.0,36.0,23.5
53115,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RD,NaN,NaN,NaN
53118,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RF,NaN,NaN,NaN
53127,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RH,217.0,50.0,23.0
53136,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RI,NaN,NaN,NaN
53142,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RP,NaN,NaN,NaN
53148,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RT,NaN,NaN,NaN


In [14]:
rows = []
for i in range(new_df1.shape[0]):
    if new_df1["SchoolCode"][new_df1.index[i]] == 112607:
        rows.append(i)
new_df1.iloc[rows]

,AcademicYear,AggregateLevel,CountryCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,CAR_RB,...,CAR_GF,CAR_GX,CAR_GZ,CAR_SE,CAR_SD,CAR_SS,CAR_SM,CAR_SF,CAR_SH,CAR_TA
0,2016-17,S,1,10017,112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,23.5,...,26.2,None,None,22.9,19.6,24.3,None,NaN,NaN,23.9


### Step 3
Write a function that generates a single DataFrame given multiple dataframes from different time periods of the same category. The resultant DataFrame should organize each school's data in chronological order.

I think we may be on different pages about what we were trying to produce in this step. I was thinking that it's organization would be as follows:

row 1: school 1, year 1

row 2: school 1, year 2

row 3: school 1, year 3

row 4: school 2, year 1

row 5: school 2, year 2, and so on.

I thought that there would be a different row per year, and we'd order the rows in the dataframe such that the schools data is grouped together and the rows are ordered chronologically. So I'm a little confused.

In [17]:
def millie_merge_df(dfs):
    master_df = pd.concat([df for df in dfs], axis=0, ignore_index=True)
    index_key = sorted(zip(master_df.index, master_df["SchoolCode"]), key=lambda x: x[1])
    sorted_indices = [item[0] for item in index_key]
    return master_df.iloc[sorted_indices]

In [108]:
def nick_merge_df(*dfs):
    masFrame = dfs[0]
    for i,frame in enumerate(dfs):
        while i < len(dfs)-1: #you don't need this while loop because you already are using a for loop that iterates
            #over the number of dataframe. The for loop will end before i >= len(dfs)-1
            arr_1 = np.array(masFrame.columns)
            i+=1 
            arr_2 = np.array(dfs[i].columns)
            comparison = arr_1 == arr_2
            equal_arrays = comparison.all()
            if equal_arrays == True:
                masterFrame = masFrame.append(dfs[i])
            else: 
                c = numpy.setxor1d(arr_1, arr_2)
                print(f"Arr_2 did not include the following columns: {c}")
                masterFrame = masFrame.append(dfs[i])
        else: #I'm confused why this else block is here... there is no corresponding if block?
            return masterFrame
            break

In [176]:
def simple_merge(dfA,dfB):
    bigFrame = pd.concat([dfA,dfB], axis=1, join='outer')
    return bigFrame

In [186]:
test_merge2 = simple_merge(new_df1, new_df2)
test_merge2.tail()

,AcademicYear,AggregateLevel,CountryCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,CAR_RB,...,CAR_GF,CAR_GX,CAR_GZ,CAR_SE,CAR_SD,CAR_SS,CAR_SM,CAR_SF,CAR_SH,CAR_TA
10012,2016-17,S,58,72751.0,6056832.0,Yuba,Wheatland,Lone Tree Elementary,All,2.6,...,11.6,None,None,22.2,17.6,15.8,None,30.0,NaN,12.2
10013,2016-17,S,58,72751.0,6056840.0,Yuba,Wheatland,Wheatland Elementary,All,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10014,2016-17,S,58,72751.0,6118806.0,Yuba,Wheatland,Wheatland Charter Academy,All,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10015,2016-17,S,58,72769.0,123570.0,Yuba,Wheatland Union High,Wheatland Community Day High,All,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10016,2016-17,S,58,72769.0,5838305.0,Yuba,Wheatland Union High,Wheatland Union High,All,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
millie_test_merge = millie_merge_df([new_df1, new_df2])

In [162]:
#I thought I already had a function that does this: drop_rows() above
test_merge = merge_df(new_df1,new_df2)
#drop empty values
misfitSchoolNames = []
for i, col in enumerate(test_merge['SchoolName']):
    if col == 'No Data' or col == 0:
        misfitSchoolNames.append(i)
test_merge = test_merge.drop(misfitSchoolNames,axis = 0)
test_merge

/var/folders/xf/8808rrps3wb3r170fzcq_6t00000gn/T/ipykernel_34243/102790977.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  masterFrame = masFrame.append(dfs[i])


,AcademicYear,AggregateLevel,CountryCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,CAR_RB,...,CAR_GF,CAR_GX,CAR_GZ,CAR_SE,CAR_SD,CAR_SS,CAR_SM,CAR_SF,CAR_SH,CAR_TA
0,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,23.5,...,26.2,None,None,22.9,19.6,24.3,None,NaN,NaN,23.9
1,2016-17,S,1,10017.0,123968.0,Alameda,Alameda County Office of Education,Community School for Creative Education,All,27.6,...,17.3,None,None,16.1,14.7,17.7,None,None,None,19.8
2,2016-17,S,1,10017.0,124172.0,Alameda,Alameda County Office of Education,Yu Ming Charter,All,21.4,...,5.6,None,None,4.3,4.3,12.1,None,None,None,4.2
3,2016-17,S,1,10017.0,125567.0,Alameda,Alameda County Office of Education,Urban Montessori Charter,All,12.7,...,13.5,None,None,14.7,15.6,17.7,None,None,NaN,12.9
4,2016-17,S,1,10017.0,130401.0,Alameda,Alameda County Office of Education,Alameda County Juvenile Hall/Court,All,7.0,...,10.0,None,None,9.5,11.9,8.8,None,11.9,8.0,8.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10008,2017-18,S,58,72751.0,6056816.0,Yuba,Wheatland,Bear River,All,0.0,...,4.7,None,None,10.3,8.3,7.0,NaN,NaN,22.2,4.5
10009,2017-18,S,58,72751.0,6056832.0,Yuba,Wheatland,Lone Tree Elementary,All,2.6,...,4.1,None,None,NaN,7.7,5.2,None,None,None,4.5
10010,2017-18,S,58,72751.0,6056840.0,Yuba,Wheatland,Wheatland Elementary,All,NaN,...,14.3,None,None,11.8,20.0,10.5,NaN,NaN,14.8,10.3
10011,2017-18,S,58,72751.0,6118806.0,Yuba,Wheatland,Wheatland Charter Academy,All,NaN,...,7.7,None,None,NaN,5.3,3.7,None,None,NaN,5.4


### Step 3
We have some schools that are outside our target grade range (9-12), so we must drop all elementary and middle schools in our DataFrames.

In [55]:
#call read_text_file function on all public school types (2021)
df_21schools = read_text_file("./txt_files/school_type_txt_files/pubschls.txt")

In [16]:
#drop rows that do not have unique school name
misfitSchoolCodes = []
for i, col in enumerate(df_21schools['School']):
    if col  == 'No Data' or col == 0:
        misfitSchoolCodes.append(i)
df_21schools = df_21schools.drop(misfitSchoolCodes,axis = 0)

In [17]:
#sort dataframe in ascending alphabetical order
def alphabetize_df(df):
    df = df.sort_values('School', ascending=True)
    return df

In [180]:
ordered_21Schools = alphabetize_df(df_21schools)
ordered_21Schools

,CDSCode,NCESDist,NCESSchool,StatusType,County,District,School,Street,StreetAbr,City,...,Virtual,Magnet,YearRoundYN,FederalDFCDistrictID,Latitude,Longitude,AdmFName,AdmLName,AdmEmail,LastUpDate
8505,29102982930121,0691023,No Data,Merged,Nevada,Nevada County Office of Education,'3R' Middle,"Error In Assign, See 6113534","Error In Assign, See 6113534",Nevada City,...,No Data,No Data,N,No Data,No Data,No Data,No Data,No Data,No Data,06/24/1999
478,1612590125856,0628050,13190,Closed,Alameda,Oakland Unified,100 Black Men of the Bay Area Community,3400 Malcolm Avenue,3400 Malcolm Ave.,Oakland,...,No Data,No Data,Y,No Data,37.745418,-122.14067,No Data,No Data,No Data,11/17/2014
11950,36679186113385,0641040,06454,Closed,San Bernardino,Victor Elementary,101 Elementary,17000 Silica Road,17000 Silica Rd.,Victorville,...,No Data,No Data,N,No Data,No Data,No Data,No Data,No Data,No Data,01/30/2009
7301,19768696023816,0601428,07146,Active,Los Angeles,Wiseburn Unified,138th Street,5403 West 138th Street,5403 West 138th St.,Hawthorne,...,N,N,N,No Data,33.906602,-118.37437,Hugo,Rios,hrios@wiseburn.org,08/21/2020
7565,21654170126912,0627720,13070,Closed,Marin,Novato Unified,180 Program,720 Diablo Avenue,720 Diablo Ave.,Novato,...,No Data,No Data,N,No Data,38.097792,-122.57617,No Data,No Data,No Data,08/05/2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7219,19753090136531,0600001,14203,Active,Los Angeles,Acton-Agua Dulce Unified,iLEAD Online,3720 Sierra Highway Suite A,3720 Sierra Hwy. Ste. A,Acton,...,F,N,Y,0601806,34.492805,-118.19593,Erin,Jones,director@ileadonline.org,10/22/2020
7223,19753090138297,0600001,14324,Active,Los Angeles,Acton-Agua Dulce Unified,iLead Agua Dulce,11311 Frascati Street,11311 Frascati St.,Agua Dulce,...,C,N,N,0602174,34.487679,-118.33247,Lisa,Latimer,director@ileadaguadulce.org,10/22/2020
9542,31103140115675,0691025,12138,Closed,Placer,Placer County Office of Education,iLearn Academy,16825 Placer Hills Road,16825 Placer Hills Rd.,Meadow Vista,...,No Data,No Data,N,No Data,38.997365,-121.02288,No Data,No Data,No Data,07/02/2013
7113,19734520120600,0633750,12477,Active,Los Angeles,Rowland Unified,iQ Academy California-Los Angeles,50 Moreland Road,50 Moreland Rd.,Simi Valley,...,F,N,N,0601964,34.275868,-118.79798,Nick,Stecken,nicstecken@iqcala.com,08/13/2020


In [62]:
def nick_drop_little_ones(df, dfSchools):
    """Index into input dataset and ordered_21schools and remove elementary and 
    middle schools from input dataframe using The Educational Instructional Level Code (EIL). 
    Below is EIL code documentation:
        A – Adult
        ELEM – Elementary
        ELEMHIGH – Elementary-High Combination
        HS – High School
        INTMIDJR – Intermediate/Middle/Junior High
        PS – Preschool
        UG – Ungraded"""
    EIL_dict = {}
    for i,school in enumerate(dfSchools['School']):
        EIL_dict[school] = dfSchools['EILCode'].iloc[i]
    EIL_codes = []
    land_o_misfits = []
    for i, school in enumerate(df["SchoolName"]):
        try:
            EIL_codes.append(EIL_dict[school])
        except:
            land_o_misfits.append(df.index[i])
    df = df.drop(land_o_misfits, axis = 0)
    print(len(EIL_codes))
    print(len(df))
    df["EILCode"] = EIL_codes
    return df

In [64]:
def millie_drop_little_ones(df, df_21schools):
    school_codes = []
    for i in range(df_21schools.shape[0]):
        school_codes.append(int(str(df_21schools["CDSCode"][i])[-7:]))
    df_21schools["SchoolCode"] = school_codes
    index_key = sorted(zip(df_21schools.index, df_21schools["SchoolCode"]), key=lambda x: x[1])
    sorted_indices = [item[0] for item in index_key]
    df_21schools = df_21schools.iloc[sorted_indices]

    i = 0
    rows_to_drop = []
    while df_21schools["SchoolCode"][df_21schools.index[i]] == 0:
        rows_to_drop.append(df_21schools.index[i])
        i += 1

    df_21schools = df_21schools.drop(rows_to_drop,axis=0)
    
    high_school_codes = []
    for i in range(df_21schools.shape[0]):
        if df_21schools["EILCode"][df_21schools.index[i]] == "HS":
            high_school_codes.append(df_21schools["SchoolCode"][df_21schools.index[i]])
            
    little_schools = []
    for i in range(df.shape[0]):
        if df["SchoolCode"][df.index[i]] not in high_school_codes:
            little_schools.append(df.index[i])
    
    df = df.drop(little_schools,axis=0)
    return df

In [65]:
final_df = millie_drop_little_ones(millie_test_merge, df_21schools)

In [66]:
final_df

,AcademicYear,AggregateLevel,CountryCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,CAR_RB,...,CAR_GF,CAR_GX,CAR_GZ,CAR_SE,CAR_SD,CAR_SS,CAR_SM,CAR_SF,CAR_SH,CAR_TA
5347,2016-17,S,32,10322,100057,Plumas,Plumas County Office of Education,Plumas County Community,All,None,...,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN
15366,2017-18,S,32,10322,100057,Plumas,Plumas County Office of Education,Plumas County Community,All,NaN,...,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN
196,2016-17,S,1,61259,100065,Alameda,Oakland Unified,Oakland Unity High,All,19,...,16.9,None,None,24.1,7.4,15,None,NaN,None,14.7
10213,2017-18,S,1,61259,100065,Alameda,Oakland Unified,Oakland Unity High,All,31.3,...,16.6,None,None,16.7,8.3,15,None,NaN,NaN,14.5
1873,2016-17,S,19,64246,100081,Los Angeles,Antelope Valley Union High,William J. (Pete) Knight High,All,21.1,...,11.8,None,None,13.1,26.7,12.3,0,27.8,32,12.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19450,2017-18,S,52,10520,6.11967e+06,Tehama,Tehama County Department of Education,Tehama eLearning Academy,All,NaN,...,29.7,None,None,NaN,26.3,29.8,None,None,47.6,28.7
4753,2016-17,S,30,66464,6.12036e+06,Orange,Capistrano Unified,Opportunities for Learning - Capistrano,All,None,...,11.2,None,None,NaN,16.7,17.9,None,NaN,25,12.6
14768,2017-18,S,30,66464,6.12036e+06,Orange,Capistrano Unified,Opportunities for Learning - Capistrano,All,NaN,...,10.1,None,None,NaN,18.5,11.7,None,NaN,NaN,7.3
495,2016-17,S,5,10058,9.01074e+06,Calaveras,Calaveras County Office of Education,Oakendell Community,All,NaN,...,None,None,None,NaN,0,4.5,None,5,None,4.5
